In [1]:
import os
import mne
import numpy as np
import pandas as pd
import itertools
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, permutation_test
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

# Fetch Data

In [2]:
main_path = './Dataset'

In [3]:
sbj_records = []
for sub in os.listdir(main_path):
    tmp = []
    for file in os.listdir(f'./Dataset/{sub}'):
        if file.split(sep='.')[1] == 'dat':
            tmp.append(mne.io.read_raw(f'./Dataset/{sub}/{file}', preload=True))
    sbj_records.append(mne.concatenate_raws(tmp))
        

Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 981199  =      0.000 ...   981.199 secs...
Event file found. Extracting Annotations from C:\Users\DFMRendering\Desktop\python\in-ear\Dataset\subject1\motor1.ceo...
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 984999  =      0.000 ...   984.999 secs...
Event file found. Extracting Annotations from C:\Users\DFMRendering\Desktop\python\in-ear\Dataset\subject1\motor2.ceo...
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 1022899  =      0.000 ...  1022.899 secs...
Event file found. Extracting Annotations from C:\Users\DFMRendering\Desktop\python\in-ear\Dataset\subject1\motor3.ceo...
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 2031599  =      0.000 ...  2031.599 secs...
Event file found. Extracting Annotations from C:\Users\DFMRendering\Desktop\python\in-ear\Dataset\subject2\motor.ceo...
Leaving device<->head transform as None (no

**resample data to 250 Hz**

In [4]:
for data in sbj_records:
    data.resample(250)

**using bandpass filter 4 - 38 Hz**

In [5]:
for data in sbj_records:
    data.filter(l_freq=4, h_freq=38, method='fir')

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.3s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.7s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.3s


Filtering raw data in 4 contiguous segments
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s


Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s


**Create epochs**

In [6]:
epochs = []
reject_criteria = dict(eeg=800e-6)

for sbj in sbj_records:
    events, event_id = mne.events_from_annotations(sbj)
    epoch = mne.Epochs(sbj, tmin=-1, tmax=2, events=events, event_id=event_id, 
                           detrend=1, event_repeated='merge', reject=reject_criteria, preload=True)
    epochs.append(epoch)


Used Annotations descriptions: ['1', '2', '700010']
Not setting metadata
242 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 242 events and 751 original time points ...
    Rejecting  epoch based on EEG : ['T9', 'FT9', 'FTT9h', 'T7', 'TP7', 'TTP7h', 'C5', 'FTT7h', 'Ft7', 'FFT7h', 'FC5', 'FCC5h', 'CCP5h', 'TPP5h', 'P7', 'P9', 'P11', 'PO11', 'PO9', 'PPO7', 'P5', 'CPP5h', 'CP3', 'C3', 'FC3', 'FFC5h', 'F5', 'F7', 'AF7', 'AFF5h', 'F3', 'FFC3h', 'FCC3h', 'C1', 'CCP3h', 'CPP3h', 'P3', 'PO3', 'POO7', 'POO9h', 'POO11h', 'I1', 'OI1', 'POO3', 'PO1', 'PPO1', 'CPP1h', 'CP1', 'CCP1h', 'FCC1h', 'FC1', 'FFC1h', 'F1', 'AF3', 'FP1', 'FPz', 'AFz', 'Fz', 'FCz', 'FCCz', 'Cz', 'CPPz', 'PPOz', 'POz', 'POOz', 'Oz', 'Iz', 'I2', 'OI2', 'POO4', 'PO2', 'PPO2', 'CPP2h', 'CP2', 'CCP2h', 'FCC2h', 'FC2', 'FFC2h', 'F2', 'AF4', 'Fp2', 'AF8', 'AFF6h', 'F4', 'FFC4h', 'FCC4h', 'C2', 'CCP4h', 'CPP4h', '

In [7]:
epochs

[<Epochs |  240 events (all good), -1 – 2 s, baseline -1 – 0 s, ~181.7 MB, data loaded,
  '1': 120
  '2': 120
  '700010': 0>,
 <Epochs |  160 events (all good), -1 – 2 s, baseline -1 – 0 s, ~121.2 MB, data loaded,
  '1': 80
  '2': 80>,
 <Epochs |  157 events (all good), -1 – 2 s, baseline -1 – 0 s, ~118.9 MB, data loaded,
  '1': 78
  '2': 78
  '800000/800001': 1>,
 <Epochs |  53 events (all good), -1 – 2 s, baseline -1 – 0 s, ~40.3 MB, data loaded,
  '1': 26
  '2': 27
  '800000/800001': 0>,
 <Epochs |  234 events (all good), -1 – 2 s, baseline -1 – 0 s, ~177.1 MB, data loaded,
  '1': 117
  '2': 117
  '800000/800001': 0>,
 <Epochs |  240 events (all good), -1 – 2 s, baseline -1 – 0 s, ~181.7 MB, data loaded,
  '1': 120
  '2': 120>]

**electrode wise standardization**

In [8]:
data_subjects = []
for sbj in epochs:
    data_subjects.append(sbj.get_data())

C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\2247543470.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_subjects.append(sbj.get_data())
C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\2247543470.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_subjects.append(sbj.get_data())
C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\2247543470.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_subjects.append(sbj.get_data())
C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\2247543470.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_subjects.append(sbj.get_data())


In [9]:
for data in data_subjects:
    for i in range(data.shape[0]):
        mean = np.mean(data[i, :])
        std = np.std(data[i, :])
        data[i, :] = (data[i, :] - mean) / std

In [10]:
for i in range(len(sbj_records)):
    sbj_records[i]._data = data_subjects[i]

In [11]:
for sbj in sbj_records:
    # Set the misc channels to EEG type
    sbj.set_channel_types({
        'RF': 'eeg', 'RB': 'eeg', 'ROD': 'eeg',
        'LF': 'eeg', 'LB': 'eeg', 'LOD': 'eeg'
    })

**There is a miss classification for sample 108, it has been labeled 5 but we just have two classes**</br>so we decided to delete it

In [12]:
np.where(epochs[2].events[..., -1] == 5)

(array([108], dtype=int64),)

In [13]:
epochs[2].drop(108)

Dropped 1 epoch: 108


Number of events,156
Events,1: 782: 78800000/800001: 0
Time range,-1.000 – 2.000 s
Baseline,-1.000 – 0.000 s


In [14]:
np.where(epochs[2].events[..., -1] == 5)

(array([], dtype=int64),)

**Re-refferencing to difference locations**

In [15]:
for sbj in epochs:
    for ch in ['LF', 'LB', 'LOD', 'LOU', 'RF', 'RB', 'ROD', 'ROU']:
        sbj.set_channel_types({ch: 'eeg'})

In [16]:
re_referenced = {
    'scalp_REF': [],
    'ear_REF': [],
    'ear_IPSI': [],
    'ear_CONTRA': [],
    'ear_EEG': []
}
scalp_channels = ["Fp2", "F7", "F3", "Fz", "F4", "F8", "FC5", "FC1", "FC2", "FC6", "T7", "C3", "Cz", "C4", "T8", "CP1", "CP2", "P7", "P3", "P4", "P8", "Oz"]
ear_channels = ['RF', 'RB', 'ROD', 'ROU', 'LF', 'LB', 'LOD', 'LOU']
for sbj in epochs:
    # Reference scalp_REF
    sbj_ref = sbj.copy().set_eeg_reference(scalp_channels)
    re_referenced['scalp_REF'].append(sbj_ref)

    # Reference ear_REF
    ear_REF = sbj.copy()
    picks = mne.pick_channels(sbj.copy().info['ch_names'], scalp_channels + ear_channels)
    rereferenced_data, _ = mne.set_eeg_reference(sbj.copy().pick_channels(scalp_channels + ear_channels), ref_channels=scalp_channels)
    ear_REF._data[:, picks, :] = rereferenced_data._data
    re_referenced['ear_REF'].append(ear_REF)

    # Reference ear_IPSI
    
    # left
    sbj_left = sbj.copy()
    picks_left = mne.pick_channels(sbj.copy().info['ch_names'], ['LF', 'LB', 'LOD', 'LOU'])
    rereferenced_data, _ = mne.set_eeg_reference(sbj.copy().pick_channels(['LF', 'LB', 'LOD', 'LOU']), ref_channels=['LOU'])
    sbj_left._data[:, picks_left, :] = rereferenced_data._data
    
    # right
    sbj_right = sbj.copy()
    picks_right = mne.pick_channels(sbj.copy().info['ch_names'], ['RF', 'RB', 'ROD', 'ROU'])
    rereferenced_data, _ = mne.set_eeg_reference(sbj.copy().pick_channels(['RF', 'RB', 'ROD', 'ROU']), ref_channels=['ROU'])
    sbj_right._data[:, picks_right, :] = rereferenced_data._data
    
    # concatinate
    combined_raw_ipsi = mne.concatenate_epochs([sbj_left, sbj_right])
    re_referenced['ear_IPSI'].append(combined_raw_ipsi)

    # Reference ear_CONTRA
    
    # left
    sbj_left_contra = sbj.copy()
    picks_left = mne.pick_channels(sbj.copy().info['ch_names'], ['LF', 'LB', 'LOD', 'LOU', 'ROU'])
    rereferenced_data, _ = mne.set_eeg_reference(sbj.copy().pick_channels(['LF', 'LB', 'LOD', 'LOU', 'ROU']), ref_channels=['ROU'])
    sbj_left_contra._data[:, picks_left, :] = rereferenced_data._data

    # right
    sbj_right_contra = sbj.copy()
    picks_right = mne.pick_channels(sbj.copy().info['ch_names'], ['RF', 'RB', 'ROD', 'ROU', 'LOU'])
    rereferenced_data, _ = mne.set_eeg_reference(sbj.copy().pick_channels(['RF', 'RB', 'ROD', 'ROU', 'LOU']), ref_channels=['LOU'])
    sbj_right_contra._data[:, picks_right, :] = rereferenced_data._data

    # concatinate
    combined_raw_contra = mne.concatenate_epochs([sbj_left_contra, sbj_right_contra])
    re_referenced['ear_CONTRA'].append(combined_raw_contra)

    # Reference ear_EEG
    sbj_ear_eeg = sbj.copy()
    picks = mne.pick_channels(sbj.copy().info['ch_names'], (['T9', 'T10'] + scalp_channels))
    rereferenced_data, _ = mne.set_eeg_reference(sbj.copy().pick_channels(['T9', 'T10'] + scalp_channels), ref_channels=scalp_channels)
    sbj_ear_eeg._data[:, picks, :] = rereferenced_data._data
    re_referenced['ear_EEG'].append(sbj_ear_eeg)


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
480 matching events found
Applying baseline correction (mode: mean)


C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\582232867.py:37: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_raw_ipsi = mne.concatenate_epochs([sbj_left, sbj_right])


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
480 matching events found
Applying baseline correction (mode: mean)


C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\582232867.py:55: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_raw_contra = mne.concatenate_epochs([sbj_left_contra, sbj_right_contra])


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
320 matching events found
Applying baseline correction (mode: mean)


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
480 matching events found
Applying baseline correction (mode: mean)


C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\582232867.py:37: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_raw_ipsi = mne.concatenate_epochs([sbj_left, sbj_right])


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Not setting metadata
480 matching events found
Applying baseline correction (mode: mean)


C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\582232867.py:55: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_raw_contra = mne.concatenate_epochs([sbj_left_contra, sbj_right_contra])


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


**Reshape data for EEGNet**

In [17]:
labels = {
    'scalp_REF': [],
    'ear_REF': [],
    'ear_IPSI': [],
    'ear_CONTRA': [],
    'ear_EEG': []
}
for key in re_referenced.keys():
    for subject in range(6):
        sample, channels, time_points = re_referenced[key][subject].get_data().shape
        labels[key].append(re_referenced[key][subject].events[..., -1])
        # reshape x data
        re_referenced[key][subject] = re_referenced[key][subject].get_data().reshape(sample, channels, time_points, 1)

C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\1712127357.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  sample, channels, time_points = re_referenced[key][subject].get_data().shape
C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\1712127357.py:13: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  re_referenced[key][subject] = re_referenced[key][subject].get_data().reshape(sample, channels, time_points, 1)
C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\1712127357.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  sample, channels, time_points = re_referenced[key][subject].get_data().shape
C:\Users\DFMRendering\AppData\Local\Temp\ipykernel_21044\1712127357.py:13: FutureWarning: The

# Model

In [18]:
def create_model(inp_shape):
    input_shape = inp_shape
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv2D(25, (1, 10), padding='valid', activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
        tf.keras.layers.DepthwiseConv2D((input_shape[0], 1), depth_multiplier=2, kernel_regularizer=tf.keras.regularizers.l2(), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((1, 3)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.SeparableConv2D(2*25, (1, 10), activation='relu', kernel_regularizer=tf.keras.regularizers.l2()),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((1, 3)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

**create and save result**

In [19]:
result = {
        'within_subject': {'scalp_REF': [],'ear_REF': [],'ear_IPSI': [],'ear_CONTRA': [],'ear_EEG': []},
        'cross_subject': {'scalp_REF': [],'ear_REF': [],'ear_IPSI': [],'ear_CONTRA': [],'ear_EEG': []},
        'individual_subject': {'scalp_REF': [],'ear_REF': [],'ear_IPSI': [],'ear_CONTRA': [],'ear_EEG': []}
    }

**Within-subject classification**

In [20]:
kf = KFold(5)

for key in re_referenced.keys():
    print(f"Method {key}:")
    for subject in range(6):
        x = re_referenced[key][subject]
        y = OneHotEncoder().fit_transform(labels[key][subject].reshape(-1, 1)).toarray()
        fold_accuracies = []
        print(f"\nSubject {subject + 1}:")
        for fold, (train_idx, test_idx) in enumerate(kf.split(x)):
            print(f"  Fold {fold + 1}:")
            # Split the data into training and testing sets
            x_train, x_test = x[train_idx], x[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Create a new model for each fold to prevent data leakage
            model = create_model(x_train.shape[1:])

            # set Callbacks
            # callback = tf.keras.callbacks.EarlyStopping(monitor='loss')
            
            # Train the model on the training set
            model.fit(x_train, y_train, epochs=150)
            
            # Test the model on the test set
            y_pred = model.predict(x_test)
            accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
            fold_accuracies.append(accuracy)
            
            print(f"Accuracy: {accuracy:.4f}")
        
        # Calculate and print the average accuracy for this subject
        avg_accuracy = np.mean(fold_accuracies)
        result['within_subject'][key].append(avg_accuracy)
        print(f"  Average accuracy for Subject {subject + 1}: {avg_accuracy:.4f}")

Method scalp_REF:

Subject 1:
  Fold 1:
Epoch 1/150


C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 3s 32ms/step - loss: 1.1727 - accuracy: 0.4427
Epoch 2/150
6/6 [==============================] - 0s 31ms/step - loss: 1.0226 - accuracy: 0.4740
Epoch 3/150
6/6 [==============================] - 0s 31ms/step - loss: 0.8611 - accuracy: 0.5469
Epoch 4/150
6/6 [==============================] - 0s 31ms/step - loss: 0.8132 - accuracy: 0.4583
Epoch 5/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7192 - accuracy: 0.4948
Epoch 6/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7363 - accuracy: 0.3958
Epoch 7/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7202 - accuracy: 0.4531
Epoch 8/150
6/6 [==============================] - 0s 32ms/step - loss: 0.7143 - accuracy: 0.4635
Epoch 9/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7132 - accuracy: 0.4740
Epoch 10/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7023 - accuracy: 0.5417
Epoch 11/150
6/6 [=============

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 1s 31ms/step - loss: 1.1106 - accuracy: 0.5156
Epoch 2/150
6/6 [==============================] - 0s 31ms/step - loss: 0.9834 - accuracy: 0.4688
Epoch 3/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7839 - accuracy: 0.5469
Epoch 4/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7928 - accuracy: 0.4583
Epoch 5/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7211 - accuracy: 0.5208
Epoch 6/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7233 - accuracy: 0.5312
Epoch 7/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7371 - accuracy: 0.4688
Epoch 8/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7175 - accuracy: 0.4792
Epoch 9/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7109 - accuracy: 0.5208
Epoch 10/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7120 - accuracy: 0.5260
Epoch 11/150
6/6 [=============

...

6/6 [==============================] - 1s 31ms/step - loss: 0.7477 - accuracy: 0.4479
Epoch 2/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7410 - accuracy: 0.4583
Epoch 3/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7234 - accuracy: 0.4948
Epoch 4/150
6/6 [==============================] - 0s 30ms/step - loss: 0.7117 - accuracy: 0.4792
Epoch 5/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7031 - accuracy: 0.5625
Epoch 6/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7130 - accuracy: 0.4948
Epoch 7/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7133 - accuracy: 0.4792
Epoch 8/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7144 - accuracy: 0.4896
Epoch 9/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7136 - accuracy: 0.4688
Epoch 10/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7103 - accuracy: 0.4688
Epoch 11/150
6/6 [=============

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 1s 31ms/step - loss: 0.7922 - accuracy: 0.4583
Epoch 2/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7396 - accuracy: 0.4531
Epoch 3/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7138 - accuracy: 0.5208
Epoch 4/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7196 - accuracy: 0.4792
Epoch 5/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7189 - accuracy: 0.5052
Epoch 6/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7140 - accuracy: 0.5365
Epoch 7/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7054 - accuracy: 0.5417
Epoch 8/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 9/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7120 - accuracy: 0.5000
Epoch 10/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7050 - accuracy: 0.5469
Epoch 11/150
6/6 [=============

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 1s 31ms/step - loss: 0.7395 - accuracy: 0.4792
Epoch 2/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7254 - accuracy: 0.5000
Epoch 3/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7173 - accuracy: 0.4792
Epoch 4/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7104 - accuracy: 0.5104
Epoch 5/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7059 - accuracy: 0.5104
Epoch 6/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7161 - accuracy: 0.4635
Epoch 7/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7099 - accuracy: 0.4583
Epoch 8/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7020 - accuracy: 0.5469
Epoch 9/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7026 - accuracy: 0.5104
Epoch 10/150
6/6 [==============================] - 0s 31ms/step - loss: 0.7072 - accuracy: 0.4844
Epoch 11/150
6/6 [=============

In [21]:
for key in result['within_subject'].keys():
    result['within_subject'][key].append(np.mean(result['within_subject'][key]))

In [22]:
ws_result = pd.DataFrame(result['within_subject']).T

In [23]:
ws_result.rename(columns={6:'Average'}, inplace=True)

In [24]:
ws_result

,0,1,2,3,4,5,Average
scalp_REF,0.845833,0.656250,0.550000,0.430909,0.837743,0.983333,0.717345
ear_REF,0.745833,0.506250,0.525806,0.507273,0.602128,0.587500,0.579132
ear_IPSI,0.897917,0.859375,0.718536,0.953680,0.638710,0.983333,0.841925
ear_CONTRA,0.754167,0.596875,0.829852,0.849784,0.780119,0.918750,0.788258
ear_EEG,0.762500,0.518750,0.512903,0.509091,0.585199,0.579167,0.577935


**cross-subject classification**

In [25]:
for key in re_referenced.keys():
    for repeat in range(6):
        # define X train and X test to fit the model
        test_subject = repeat
        train_subjects = [i for i in range(6) if i != test_subject]
        x_train = np.concatenate([re_referenced[key][i] for i in train_subjects], axis=0)
        y_train = OneHotEncoder().fit_transform(np.concatenate([labels[key][i] for i in train_subjects], axis=0).reshape(-1, 1)).toarray()
        x_test = re_referenced[key][test_subject]
        y_test = OneHotEncoder().fit_transform(labels[key][test_subject].reshape(-1, 1)).toarray()

        # create model
        model = create_model(x_train.shape[1:])
        # fit model
        model.fit(x_train, y_train, epochs=150)
        # Test the model on the test set
        y_pred = model.predict(x_test)
        accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
        result['cross_subject'][key].append(accuracy)

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
27/27 [==============================] - 1s 31ms/step - loss: 1.3336 - accuracy: 0.5136
Epoch 2/150
27/27 [==============================] - 1s 30ms/step - loss: 1.0100 - accuracy: 0.4994
Epoch 3/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7548 - accuracy: 0.5089
Epoch 4/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7223 - accuracy: 0.4638
Epoch 5/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7189 - accuracy: 0.4698
Epoch 6/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7115 - accuracy: 0.5113
Epoch 7/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7118 - accuracy: 0.4935
Epoch 8/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7130 - accuracy: 0.4828
Epoch 9/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7086 - accuracy: 0.5314
Epoch 10/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7087 - accuracy: 0.5445

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
29/29 [==============================] - 1s 33ms/step - loss: 1.2973 - accuracy: 0.5125
Epoch 2/150
29/29 [==============================] - 1s 31ms/step - loss: 1.2509 - accuracy: 0.4702
Epoch 3/150
29/29 [==============================] - 1s 31ms/step - loss: 0.9588 - accuracy: 0.4821
Epoch 4/150
29/29 [==============================] - 1s 31ms/step - loss: 0.7324 - accuracy: 0.4789
Epoch 5/150
29/29 [==============================] - 1s 31ms/step - loss: 0.7142 - accuracy: 0.4897
Epoch 6/150
29/29 [==============================] - 1s 31ms/step - loss: 0.7151 - accuracy: 0.4962
Epoch 7/150
29/29 [==============================] - 1s 31ms/step - loss: 0.7133 - accuracy: 0.4810
Epoch 8/150
29/29 [==============================] - 1s 31ms/step - loss: 0.7108 - accuracy: 0.4854
Epoch 9/150
29/29 [==============================] - 1s 31ms/step - loss: 0.7123 - accuracy: 0.5016
Epoch 10/150
29/29 [==============================] - 1s 31ms/step - loss: 0.7103 - accuracy: 0.4810

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

Epoch 1/150
33/33 [==============================] - 2s 30ms/step - loss: 1.3257 - accuracy: 0.4825
Epoch 2/150
33/33 [==============================] - 1s 30ms/step - loss: 1.1873 - accuracy: 0.5194
Epoch 3/150
33/33 [==============================] - 1s 30ms/step - loss: 0.9581 - accuracy: 0.5078
Epoch 4/150
33/33 [==============================] - 1s 30ms/step - loss: 0.7351 - accuracy: 0.4893
Epoch 5/150
33/33 [==============================] - 1s 30ms/step - loss: 0.7165 - accuracy: 0.4893
Epoch 6/150
33/33 [==============================] - 1s 30ms/step - loss: 0.7236 - accuracy: 0.4893
Epoch 7/150
33/33 [==============================] - 1s 30ms/step - loss: 0.7156 - accuracy: 0.4990
Epoch 8/150
33/33 [==============================] - 1s 30ms/step - loss: 0.7141 - accuracy: 0.5039
Epoch 9/150
33/33 [==============================] - 1s 30ms/step - loss: 0.7130 - accuracy: 0.5068
Epoch 10/150
33/33 [==============================] - 1s 31ms/step - loss: 0.7153 - accuracy: 0.5194

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
27/27 [==============================] - 2s 31ms/step - loss: 1.2206 - accuracy: 0.4947
Epoch 2/150
27/27 [==============================] - 1s 31ms/step - loss: 0.8975 - accuracy: 0.5088
Epoch 3/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7169 - accuracy: 0.5065
Epoch 4/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7084 - accuracy: 0.5406
Epoch 5/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7122 - accuracy: 0.5206
Epoch 6/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7116 - accuracy: 0.5206
Epoch 7/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7144 - accuracy: 0.4700
Epoch 8/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7150 - accuracy: 0.4723
Epoch 9/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7095 - accuracy: 0.5088
Epoch 10/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7129 - accuracy: 0.4900

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
27/27 [==============================] - 1s 30ms/step - loss: 1.4839 - accuracy: 0.4899
Epoch 2/150
27/27 [==============================] - 1s 30ms/step - loss: 1.3936 - accuracy: 0.4864
Epoch 3/150
27/27 [==============================] - 1s 31ms/step - loss: 1.1615 - accuracy: 0.5065
Epoch 4/150
27/27 [==============================] - 1s 30ms/step - loss: 0.9867 - accuracy: 0.4816
Epoch 5/150
27/27 [==============================] - 1s 30ms/step - loss: 0.8823 - accuracy: 0.4733
Epoch 6/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7190 - accuracy: 0.4970
Epoch 7/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7126 - accuracy: 0.5006
Epoch 8/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7109 - accuracy: 0.5160
Epoch 9/150
27/27 [==============================] - 1s 30ms/step - loss: 0.7134 - accuracy: 0.4840
Epoch 10/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7117 - accuracy: 0.4923

In [26]:
cs_result = pd.DataFrame(result['cross_subject']).T

In [29]:
for key in result['cross_subject'].keys():
    result['cross_subject'][key].append(np.mean(result['cross_subject'][key]))

In [30]:
ws_result.rename(columns={6:'Average'}, inplace=True)

In [31]:
ws_result

,0,1,2,3,4,5,Average
scalp_REF,0.845833,0.656250,0.550000,0.430909,0.837743,0.983333,0.717345
ear_REF,0.745833,0.506250,0.525806,0.507273,0.602128,0.587500,0.579132
ear_IPSI,0.897917,0.859375,0.718536,0.953680,0.638710,0.983333,0.841925
ear_CONTRA,0.754167,0.596875,0.829852,0.849784,0.780119,0.918750,0.788258
ear_EEG,0.762500,0.518750,0.512903,0.509091,0.585199,0.579167,0.577935


**individual subject retraining**

In [20]:
for key in re_referenced.keys():
    for repeat in range(6):
        # define X train and X test to fit the model
        test_subject = repeat
        train_subjects = [i for i in range(6) if i != test_subject]
        x_train = np.concatenate([re_referenced[key][i] for i in train_subjects], axis=0)
        y_train = OneHotEncoder().fit_transform(np.concatenate([labels[key][i] for i in train_subjects], axis=0).reshape(-1, 1)).toarray()
        y_test = OneHotEncoder().fit_transform(labels[key][test_subject].reshape(-1, 1)).toarray()
        x_retrain_train, x_retrain_test, y_retrain_train, y_retrain_test = train_test_split(re_referenced[key][test_subject], y_test, test_size=0.2)

        # create model
        model = create_model(x_train.shape[1:])
        # fit model
        model.fit(x_train, y_train, epochs=150)
        # Retrain on the test subject's data
        model.fit(x_retrain_train, y_retrain_train, epochs=70, verbose=0)
        # Test the model on the test set
        y_pred = model.predict(x_retrain_test)
        accuracy = accuracy_score(np.argmax(y_retrain_test, axis=1), np.argmax(y_pred, axis=1))
        result['individual_subject'][key].append(accuracy)

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
27/27 [==============================] - 7s 36ms/step - loss: 1.3846 - accuracy: 0.4757
Epoch 2/150
27/27 [==============================] - 1s 33ms/step - loss: 1.1043 - accuracy: 0.5077
Epoch 3/150
27/27 [==============================] - 1s 31ms/step - loss: 0.9392 - accuracy: 0.5065
Epoch 4/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7291 - accuracy: 0.5006
Epoch 5/150
27/27 [==============================] - 1s 32ms/step - loss: 0.7115 - accuracy: 0.5113
Epoch 6/150
27/27 [==============================] - 1s 32ms/step - loss: 0.7169 - accuracy: 0.4970
Epoch 7/150
27/27 [==============================] - 1s 32ms/step - loss: 0.7119 - accuracy: 0.4899
Epoch 8/150
27/27 [==============================] - 1s 32ms/step - loss: 0.7102 - accuracy: 0.5006
Epoch 9/150
27/27 [==============================] - 1s 32ms/step - loss: 0.7152 - accuracy: 0.4899
Epoch 10/150
27/27 [==============================] - 1s 32ms/step - loss: 0.7071 - accuracy: 0.5267

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
29/29 [==============================] - 2s 34ms/step - loss: 1.3754 - accuracy: 0.5027
Epoch 2/150
29/29 [==============================] - 1s 32ms/step - loss: 1.3093 - accuracy: 0.4973
Epoch 3/150
29/29 [==============================] - 1s 32ms/step - loss: 1.2298 - accuracy: 0.5114
Epoch 4/150
29/29 [==============================] - 1s 32ms/step - loss: 0.9729 - accuracy: 0.5081
Epoch 5/150
29/29 [==============================] - 1s 32ms/step - loss: 0.7205 - accuracy: 0.5081
Epoch 6/150
29/29 [==============================] - 1s 32ms/step - loss: 0.7147 - accuracy: 0.4984
Epoch 7/150
29/29 [==============================] - 1s 32ms/step - loss: 0.7118 - accuracy: 0.4930
Epoch 8/150
29/29 [==============================] - 1s 32ms/step - loss: 0.7108 - accuracy: 0.5125
Epoch 9/150
29/29 [==============================] - 1s 32ms/step - loss: 0.7113 - accuracy: 0.4930
Epoch 10/150
29/29 [==============================] - 1s 32ms/step - loss: 0.7090 - accuracy: 0.4984

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

Epoch 1/150
33/33 [==============================] - 2s 31ms/step - loss: 1.2559 - accuracy: 0.4932
Epoch 2/150
33/33 [==============================] - 1s 31ms/step - loss: 0.9911 - accuracy: 0.4748
Epoch 3/150
33/33 [==============================] - 1s 31ms/step - loss: 0.7371 - accuracy: 0.5165
Epoch 4/150
33/33 [==============================] - 1s 31ms/step - loss: 0.7217 - accuracy: 0.4718
Epoch 5/150
33/33 [==============================] - 1s 31ms/step - loss: 0.7140 - accuracy: 0.4767
Epoch 6/150
33/33 [==============================] - 1s 31ms/step - loss: 0.7137 - accuracy: 0.4835
Epoch 7/150
33/33 [==============================] - 1s 31ms/step - loss: 0.7096 - accuracy: 0.5029
Epoch 8/150
33/33 [==============================] - 1s 32ms/step - loss: 0.7134 - accuracy: 0.4699
Epoch 9/150
33/33 [==============================] - 1s 32ms/step - loss: 0.7101 - accuracy: 0.5107
Epoch 10/150
33/33 [==============================] - 1s 31ms/step - loss: 0.7083 - accuracy: 0.4942

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
27/27 [==============================] - 1s 31ms/step - loss: 1.1911 - accuracy: 0.4841
Epoch 2/150
27/27 [==============================] - 1s 31ms/step - loss: 0.8133 - accuracy: 0.5053
Epoch 3/150
27/27 [==============================] - 1s 32ms/step - loss: 0.7271 - accuracy: 0.4617
Epoch 4/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7151 - accuracy: 0.5147
Epoch 5/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7138 - accuracy: 0.5053
Epoch 6/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7113 - accuracy: 0.4947
Epoch 7/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7114 - accuracy: 0.5194
Epoch 8/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7182 - accuracy: 0.4959
Epoch 9/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7139 - accuracy: 0.5065
Epoch 10/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7145 - accuracy: 0.4794

C:\Users\DFMRendering\Desktop\python\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150
27/27 [==============================] - 1s 31ms/step - loss: 1.6515 - accuracy: 0.4816
Epoch 2/150
27/27 [==============================] - 1s 31ms/step - loss: 1.2796 - accuracy: 0.5125
Epoch 3/150
27/27 [==============================] - 1s 31ms/step - loss: 1.4219 - accuracy: 0.4721
Epoch 4/150
27/27 [==============================] - 1s 31ms/step - loss: 1.2794 - accuracy: 0.5148
Epoch 5/150
27/27 [==============================] - 1s 31ms/step - loss: 1.0195 - accuracy: 0.5077
Epoch 6/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7233 - accuracy: 0.5065
Epoch 7/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7164 - accuracy: 0.5184
Epoch 8/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7195 - accuracy: 0.5030
Epoch 9/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7147 - accuracy: 0.5030
Epoch 10/150
27/27 [==============================] - 1s 31ms/step - loss: 0.7194 - accuracy: 0.4994

In [28]:
is_result = pd.DataFrame(result['individual_subject']).T

In [22]:
for key in result['individual_subject'].keys():
    result['individual_subject'][key].append(np.mean(result['individual_subject'][key]))

In [29]:
is_result.rename(columns={6:'Average'}, inplace=True)

In [30]:
is_result

,0,1,2,3,4,5,Average
scalp_REF,0.520833,0.843750,0.562500,0.454545,0.574468,1.000000,0.659349
ear_REF,0.833333,0.687500,0.406250,0.727273,0.808511,0.437500,0.650061
ear_IPSI,0.583333,0.968750,0.539683,0.772727,0.989362,1.000000,0.808976
ear_CONTRA,0.979167,0.796875,0.841270,0.454545,0.946809,1.000000,0.836444
ear_EEG,0.479167,0.531250,0.906250,0.363636,0.829787,0.979167,0.681543
